In [2]:
import pandas as pd
import glob
import yfinance as yf
import requests
import pyxirr

Preprocessing

In [4]:
file_paths = glob.glob('stock_dataset/*.csv')

In [5]:
dfs = []
for fp in file_paths:
    df = pd.read_csv(fp)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

In [6]:
# Quantity to numeric and sorting by date/time
merged_df['Quantity'] = pd.to_numeric(merged_df['Quantity'], errors='coerce')
merged_df = merged_df.sort_values(by='Date/Time').reset_index(drop=True)

In [7]:
# Get Split Details
symbols = [
    "AAPL", "AMZN", "AXP", "C6L", "CMG", "GOOG", "GOOGL", "GPRO", "IBIT", "MARA",
    "MSFT", "NET", "NU", "NVDA", "ORCL", "SE", "SPY", "TSLA"
]

split_details = {}
start_date = pd.to_datetime("2023-06-01")

for symbol in symbols:
    ticker = yf.Ticker(symbol)
    splits = ticker.splits
    if isinstance(splits.index, pd.DatetimeIndex):
        splits.index = splits.index.tz_convert(None) if splits.index.tz else splits.index
        splits_after = splits[splits.index > start_date]
        if not splits_after.empty:
            print(symbol, splits_after)
            split_details[symbol] = splits_after
        else:
            split_details[symbol] = "No split data after start date"
    else:
        print(f"{symbol}: No split data or index not datetime")
        split_details[symbol] = "No split data or index not datetime"

HTTP Error 404: 
$C6L: possibly delisted; no timezone found


C6L: No split data or index not datetime
CMG Date
2024-06-26 04:00:00    50.0
Name: Stock Splits, dtype: float64
NVDA Date
2024-06-10 04:00:00    10.0
Name: Stock Splits, dtype: float64


In [8]:
# Adjust according to splits
df_splits = merged_df.copy()
for symbol, splits in split_details.items():
    if isinstance(splits, pd.Series):
        for split_date, split_ratio in splits.items():
            split_date = pd.to_datetime(split_date).replace(tzinfo=None)
            mask = (
                (df_splits['Symbol'] == symbol) &
                (pd.to_datetime(df_splits['Date/Time'], errors='coerce') < split_date)
            )
            matching_rows = df_splits.loc[mask, ['Date/Time']]
            print(f"{symbol} | Split Date: {split_date.date()} | Matches: {mask.sum()}")
            df_splits.loc[mask, 'Quantity'] = df_splits.loc[mask, 'Quantity'] * split_ratio
            df_splits.loc[mask, 'T. Price'] = df_splits.loc[mask, 'T. Price'] / split_ratio
            df_splits.loc[mask, 'Proceeds'] = (
                df_splits.loc[mask, 'Quantity'] * df_splits.loc[mask, 'T. Price'] * 
                (-1 if (merged_df.loc[mask, 'Proceeds'] < 0).any() else 1)
            )

CMG | Split Date: 2024-06-26 | Matches: 1
NVDA | Split Date: 2024-06-10 | Matches: 17


In [9]:
# Compare Quantity, T. Price, and Proceeds for NVDA rows before split date in both DataFrames
split_date = pd.to_datetime('2024-06-10')
mask = (pd.to_datetime(merged_df['Date/Time'], errors='coerce') < split_date) & (merged_df['Symbol'] == 'NVDA')

comparison = pd.DataFrame({
    'Quantity_original': merged_df.loc[mask, 'Quantity'],
    'T. Price_original': merged_df.loc[mask, 'T. Price'],
    'Proceeds_original': merged_df.loc[mask, 'Proceeds'],
    'Quantity_split': df_splits.loc[mask, 'Quantity'],
    'T. Price_split': df_splits.loc[mask, 'T. Price'],
    'Proceeds_split': df_splits.loc[mask, 'Proceeds'],
})

comparison.iloc[0:3]

,Quantity_original,T. Price_original,Proceeds_original,Quantity_split,T. Price_split,Proceeds_split
1,40.0,380.0,-15200.0,400.0,38.0,-15200.0
6,20.0,449.0,-8980.0,200.0,44.9,-8980.0
8,-20.0,502.0,10040.0,-200.0,50.2,10040.0


In [ ]:
# Get the inr and sgd value for usd for uniques date
trade_dates = pd.to_datetime(df_splits['Date/Time'], errors='coerce').dt.date.unique()
currency_data = []
for date in trade_dates:
    url = f"https://api.frankfurter.app/{date}?from=USD&to=INR,SGD"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        rates = data.get('rates', {})
        currency_data.append({
            'Date': date,
            'USDINR': rates.get('INR', None),
            'USDSGD': rates.get('SGD', None)
        })
    else:
        print('else')
        currency_data.append({
            'Date': date,
            'USDINR': None,
            'USDSGD': None
        })
currency_rates = pd.DataFrame(currency_data)
currency_rates.to_csv('currency_dataset/currency_rates.csv', index=False)

In [11]:
df_currency = df_splits.copy()

In [13]:
# Load currency rates
currency_rates = pd.read_csv('currency_dataset/currency_rates.csv')
currency_rates['Date'] = pd.to_datetime(currency_rates['Date']).dt.date

# Prepare df_splits for merging
df_currency['Date'] = pd.to_datetime(df_currency['Date/Time'], errors='coerce').dt.date

# Merge currency rates into df_splits
df_currency = pd.merge(df_currency, currency_rates, on='Date', how='left')

# Compute transaction price in INR and SGD
df_currency['Price_INR'] = df_currency['T. Price'] * df_currency['USDINR']
df_currency['Price_SGD'] = df_currency['T. Price'] * df_currency['USDSGD']

In [14]:
df_splits_close = df_currency.copy()

In [ ]:
# Get split adjusted historical prices
results = []
for idx, row in df_splits_close.iterrows():
    symbol = row['Symbol']
    date_str = pd.to_datetime(row['Date/Time'], errors='coerce').strftime('%Y-%m-%d') if not pd.isna(row['Date/Time']) else None
    close_price = None
    if symbol and date_str:
        try:
            data = yf.download(
                symbol,
                start=date_str,
                end=(pd.to_datetime(date_str) + pd.Timedelta(days=1)).strftime('%Y-%m-%d'),
                auto_adjust=True
            )
            # Get the close for the exact date
            if not data.empty and 'Close' in data.columns:
                close_row = data.loc[data.index.normalize() == pd.to_datetime(date_str)]
                if not close_row.empty:
                    close_price = float(close_row['Close'].iloc[0]) 
        except Exception as e:
            print(f"Error for {symbol} on {date_str}: {e}")
    results.append({
        'Symbol': symbol,
        'Date/Time': row['Date/Time'],
        'Split_Adjusted_Close': close_price
    })

# Create a new DataFrame with the results
df_closing_prices = pd.DataFrame(results)
print(df_closing_prices.head())

In [106]:
# Merge closing prices from df_closing_prices into df_splits_close
df_splits_close = pd.merge(
    df_splits_close,
    df_closing_prices[['Symbol', 'Date/Time', 'Split_Adjusted_Close']],
    on=['Symbol', 'Date/Time'],
    how='left'
)

In [ ]:
df_splits_close.to_csv('final_dataset.csv', index=False)